# **Planet Imagery**

Planet achieved [**Mission one**](https://www.planet.com/pulse/mission-1), **imaging the entire earth every single day**. Having such an ambitious mission and achieveing it meant meant that can now analyze time series datasets over weekly and bi weekly periods and understand questions that require dynamic analysis such as vegetation patters, climate variability and changes in hydrological cycles. It generated the single largest trove of high frequency dataset in existence with an unprecedented cadence or repeat over areas.

To think of Planet products you have to understand two terms as thought they live in a hierarchy
Items and Assets
![planet-products](https://user-images.githubusercontent.com/6677629/66722158-0e8a6c80-edbf-11e9-95c2-3160e1598f7c.JPG)
* **Item type** almost refers exclusively to a family of satellite or sensor types so PlanetScope, RapidEye, Skysat, Landsat and so on are all item types. These are model definitions based on the type of sensor you are utilizing for performing any type of analysis.

* **Asset type** are types of item derivatives or data types that you are actually utilizing for example analytic, analytic_sr, analytic_xml, visual and so on. These allow you to choose the type of actual data that you are able to download including the type and level of preprocesing that has been applied to it.

To read more about Planet items and assets, [try this page](https://developers.planet.com/docs/api/items-assets/)



**Initialize the Planet Client**

Before we get anything started you should initialize the planet client. In this this required your email address and password associated with your Planet explorer account. If you are using Google colab you can remove the '#' symbol before ```!planet init```. Else open a new terminal in mybinder and use the method below. Then go back to the notebook

![initialize](https://user-images.githubusercontent.com/6677629/66725367-22959480-ede6-11e9-90e6-a4f53c33a95a.gif)


In [0]:
# Test planet installation
try:
  import planet
except ImportError:
  !pip install planet==1.3.2

#If using Colab
#!planet init

**Upload your AOI**

Visit [geojson.io](http://geojson.io) and draw your AOI. You will use this geometry to search for images that intersect your area and download them eventually. While search does not depend on your quota, download does. A useful rule of thumb, do not draw an extremely large AOI response might take a long time and fail. Avoid choosing a whole country or continent scale AOI :)

**Check your Quota**

Before you start work on this notebook check your quota in Sqkm to see if you can download your imagery. We are going to download only one image and then going to call this function again



In [0]:
import requests
import sys
from planet.api.auth import find_api_key


# Function to get user's quota
def planet_quota():
    # Get API Key: Requires user to have initialized Planet CLI
    try:
        api_key = find_api_key()
    except Exception as e:
        print(e)
        print('Failed to get Planet Key: Try planet init')
        sys.exit()
    '''Print allocation and remaining quota in Sqkm.'''
    try:
        main = requests.get('https://api.planet.com/auth/v1/experimental/public/my/subscriptions', auth=(api_key, ''))
        if main.status_code == 200:
            content = main.json()
            for item_id in content:
                print(" ")
                print(
                    'Subscription ID: %s'
                    % item_id['id'])
                print(
                    'Plan ID: %s'
                    % item_id['plan_id'])
                print(
                    'Allocation Name: %s'
                    % item_id['organization']['name'])
                print(
                    'Allocation active from: %s'
                    % item_id['active_from'].split("T")[0])
                print(
                    'Quota Enabled: %s'
                    % item_id['quota_enabled'])
                print(
                    'Total Quota in SqKm: %s'
                    % item_id['quota_sqkm'])
                print(
                    'Total Quota used: %s'
                    % item_id['quota_used'])
                if (item_id['quota_sqkm'])is not None:
                    leftquota = (float(
                        item_id['quota_sqkm'] - float(item_id['quota_used'])))
                    print(
                        'Remaining Quota in SqKm: %s' % leftquota)
                else:
                    print('No Quota Allocated')
                print('')
        elif main.status_code == 500:
            print('Temporary issue: Try again')
        else:
            print('Failed with exception code: ' + str(
                main.status_code))

    except IOError:
        print('Initialize client or provide API Key')


In [0]:
planet_quota()

**Planet Stats**

Stats is an extremely useful too and is often when I start planning for my datasets. In this case right click on the geojson file you just uploaded and choose *copy path*. We will use this geometry file.

The parameters that enter into the setup include


1.   **interval**- month, week, year and so on
2.   **item-type** -PSScene4Band, PSOrthoTile, REOrthoTile and so on
3. **asset-type**- This is not necessary but you can provide an asset type like analytic, analytic_sr and so on

To read more about Planet items and assets, [try this page](https://developers.planet.com/docs/api/items-assets/)



In [0]:
!planet data stats --interval month --item-type PSScene4Band --asset-type analytic --geom /aoi.geojson

**Generate Planet Data Footprints**

The next thing I like to do, I want to see where these images are, what are the footprints of these images. This is also something that does not use your quota, but requires you to have an active planet account and a quota subscription. We are writing the metadata along with the geometry as a geojson file and upload it back to [geojson.io](http://geojson.io)

We use the flag **--limit** to specify the number of images we want to look for. Here I am specifying *100* images

In [0]:
!planet data search --item-type PSScene4Band --asset-type analytic --limit 100 --geom /aoi.geojson >/fp.geojson

![Footprint_metadata](https://user-images.githubusercontent.com/6677629/66720585-8d79a800-edb3-11e9-81da-9c7c1444b957.PNG)

**Downloading Planet Data**

This gets more interesting and will use your quota. Now before you get into the weeds about the planet command line tool that we have been exploring throughout the tutorial. Here is a medium article **[Demystifying Planet’s Command Line Tool](https://medium.com/planet-stories/demystifying-planets-command-line-tool-660870b32317)**

This should help you get to more advanced things. For this case we are going to download one image, we are going to set a start and end date and also use a cloud cover. We definitely want a clean imagery without clouds. So the things we are going to introduce are (here gt and lt specifies greater than and less than). Here cloud cover can be replaced with any other numeric metadata
1. Start Date --date acquired gt "2018–01–01"  
2. End date --date acquired lt "2018–12–31"
3. Cloud cover --range cloud_cover lt "0.2"


You can copy and paste this in your terminal window. The client has issues with piping the response so we use subprocess to download the imagery. Planet imagery is not active and needs time to be activated and then the download process can begin. So this can take some time even if we are downloading only a single image

In [0]:
import subprocess
try:
  subprocess.call("planet data download --item-type PSScene4Band --asset-type analytic --date acquired gt 2018-01-01 --date acquired lt 2018-12-31 --range cloud_cover lt 0.2 --limit 1 --geom /aoi.geojson --dest /usr/planet",shell=True)
except Exception as e:
  print(e)

Check the quota again it should have reduced by some amount.

In [0]:
planet_quota()

**Citing Planet Data**

From a concept in our garage, to operating the largest fleet of Earth-imaging satellites, many people have invested time and energy in developing and enabling access to Planet’s unique data feed. Please cite Planet when using our imagery and tools.

To cite Planet data in publications, please use the following:


*Planet Team (2017). Planet Application Program Interface: In Space for Life on Earth. San Francisco, CA. https://api.planet.com.*